In [1]:
import argilla as rg
import pandas as pd
import numpy as np

In [ ]:
rg.init(
    api_url="http://localhost:6900/",
    api_key="admin.apikey",
    workspace="admin",
    extra_headers={"X-Argilla-Workspace": "my_connection_headers"}
)

In [2]:
# Load the dataset
ushahidi_df = pd.read_csv("data/ushahidi.csv")
ushahidi_df.shape

/var/folders/pz/qy3yq1t97s7g1m5s3hbsqjpc0000gn/T/ipykernel_36032/561396766.py:2: DtypeWarning: Columns (4,8,10,11,12,13,16,17,18,19,20,21,22,23,24,25,26,28,29,30,31,32,33,34,37,38,39,40,41,42,44,45,46,47,49,50,51,52,53,54,55,58,59,60,61,62,63,64,65,66,67,68,70,71,72,73,74,77,78,79,80,81,82,83,84,85,86,87,89,90,91,92,93,94,95,98,99,100,101,102,103,104,105,106,107,108,110,111,112,113,114,115,116,119,120,121,123,124,125,126,128,129,131,132,133,134,135,136,137,140,141,142,143,144,145,146,147,148,149,150,152,153,154,155,156,157,158,161,162,163,164,165,166,167,168,169,170,172,173,175,176,177,178,179,182,183,184,185,186,187,188,189,190,192,193,194,195,196,197,200,201,202,203,204,206,207,208,209,212,213,214,215,218,219,220,221,222,223,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,244,245,246,251,255,258,259) have mixed types. Specify dtype option on import or set low_memory=False.
  ushahidi_df = pd.read_csv("data/ushahidi.csv")


(102387, 260)

In [3]:
# What if I create a function that takes a list of duplicated columns then for each column, finds the columns related to it and creates a new column with the cleaned values
#Dictionary to contain column name as key and the new column name to create 
#dp prefix before column name is used as shorthand for deduplicated
duplicated_cols_to_be_cleaned = { 
    "Title":"dp_title",
    "Description":"dp_description",
    "Location.lat": "dp_location_lat", 
    "Location.lon": "dp_location_long",
    "Nature of this incident":"dp_nature_of_this_incident",
    "Verification Status":"dp_verificiation_status",
    "Media Source":"dp_media_source",
    "Translation Status": "dp_translation_status",
    "Description Translation": "dp_description_translation"
}

def deduplicate_cols(dp_dict:dict, df:pd.DataFrame):
    def clean_row(row, dup_cols):
        val = np.nan
        for col in dup_cols:
            if pd.isnull(row[col]) != True:
                val = row[col]
        return val 

    for duplicted_col_name in dp_dict.keys():
        # Get all the list of columns that match the duplicated column name 
        dup_cols = df.columns[df.columns.str.contains(pat = duplicted_col_name)]
        # Create a new column that deduplicates the columns
        df[dp_dict[duplicted_col_name]] = df.apply(lambda row: clean_row(row, dup_cols), axis=1)
    
    return df


In [4]:
ushahidi_df = deduplicate_cols(duplicated_cols_to_be_cleaned, ushahidi_df)

In [5]:
labels = ['Opinions', 'Political Rallies', 'Positive Events', 'Security Issues', 'Media Reports',
       'Voting Issues', 'Counting and Results', 'Staffing Issues', 'Polling Station Administration', 'Unstructured', 'Other']

In [6]:
data_df = ushahidi_df[ushahidi_df['Survey'].isin(labels)]

In [7]:
deduplicated_cols = list(duplicated_cols_to_be_cleaned.values())

cols_of_interest = ["Post ID", "Survey", "Post Status", "Created (UTC)", "Updated (UTC)", "Post Date (UTC)", 
                    "Source", "Data Source ID", "Source"]

cols_of_interest = cols_of_interest +  deduplicated_cols

In [8]:
filtered_df = data_df[["Post ID","Survey","dp_description","Post Status", "Post Date (UTC)","dp_nature_of_this_incident","dp_location_lat","dp_location_long"]]

In [9]:
filtered_df.shape

(100277, 8)

In [10]:
filtered_df.head()

,Post ID,Survey,dp_description,Post Status,Post Date (UTC),dp_nature_of_this_incident,dp_location_lat,dp_location_long
1,4.0,Polling Station Administration,https://twitter.com/1146124540055052289/status...,published,2022-06-27 13:45:01,Polling station logistical issues,-1.303169,36.826061
2,5.0,Unstructured,NaN,draft,2022-06-27 13:44:20,NaN,NaN,NaN
3,6.0,Polling Station Administration,https://twitter.com/1510781350902706180/status...,archived,2022-06-27 13:25:51,Polling station logistical issues,-1.283253,36.817245
4,7.0,Unstructured,NaN,draft,2022-06-27 12:56:55,NaN,NaN,NaN
5,8.0,Unstructured,NaN,draft,2022-06-27 13:45:01,NaN,NaN,NaN


In [11]:
filtered_df["Post Date (UTC)"].info()

<class 'pandas.core.series.Series'>
Int64Index: 100277 entries, 1 to 102379
Series name: Post Date (UTC)
Non-Null Count   Dtype 
--------------   ----- 
100277 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


In [12]:
uchaguzi_df = pd.read_csv("data/Uchaguzi August 2017.csv")
uchaguzi_df.shape

/var/folders/pz/qy3yq1t97s7g1m5s3hbsqjpc0000gn/T/ipykernel_36032/2918578088.py:1: DtypeWarning: Columns (4,9,10,11,14,15,16,17,18,19,20,21,22,23,24,26,27,28,29,30,31,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,52,53,54,56,57,58,59,60,61,62,64,65,66,67,70,71,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,90,91,92,94,95,96,97,98,99,100,101,103,104,105,106,107,108,111,112,113,115,116,117,118,119,120,121,122,124,125,126,127,128,129,132,133,134,135,136,138,139,140,141,142,143,145,146,147,148,149,151,152,153,154,155,158,159,160,161,162,163,164,165,166,167,168,170,172,173,174,175,176,179,180,181,182,183,184,185,186,187,188,189,191,192,193,194,199,200,201,202,203,205,206,207,209,210,211,213,214,215,217,218,219,220,221,223,225,227,229) have mixed types. Specify dtype option on import or set low_memory=False.
  uchaguzi_df = pd.read_csv("data/Uchaguzi August 2017.csv")


(49122, 230)

In [13]:
uchaguzi_rerun_df = pd.read_csv("data/Uchaguzi October 2017.csv")
uchaguzi_rerun_df.shape

/var/folders/pz/qy3yq1t97s7g1m5s3hbsqjpc0000gn/T/ipykernel_36032/2824090327.py:1: DtypeWarning: Columns (16,25,43,44,102,103,113,119,121,123,126,135,145,146,148,157,163,199) have mixed types. Specify dtype option on import or set low_memory=False.
  uchaguzi_rerun_df = pd.read_csv("data/Uchaguzi October 2017.csv")


(7617, 204)

In [14]:
# Drop all columns that are fully null
print(f"Before dropping columns that are null : {uchaguzi_rerun_df.shape}")
uchaguzi_df = uchaguzi_df.dropna(axis=1, how='all')
print(f"After dropping columns that are null : {uchaguzi_rerun_df.shape}")

Before dropping columns that are null : (7617, 204)
After dropping columns that are null : (7617, 204)


In [15]:
# Drop all columns that are fully null
print(f"Before dropping columns that are null : {uchaguzi_df.shape}")
uchaguzi_df = uchaguzi_df.dropna(axis=1, how='all')
print(f"After dropping columns that are null : {uchaguzi_df.shape}")

Before dropping columns that are null : (49122, 213)
After dropping columns that are null : (49122, 213)


In [16]:
uchaguzi_df = deduplicate_cols(duplicated_cols_to_be_cleaned, uchaguzi_df)

In [17]:
uchaguzi_rerun_df = deduplicate_cols(duplicated_cols_to_be_cleaned, uchaguzi_rerun_df)

In [ ]:
uchaguzi_rerun_df['']

In [18]:
uchaguzi_rerun_df.head()

,Post ID,Survey,Post Status,Created (UTC),Updated (UTC),Post Date (UTC),Contact ID,Contact,Unstructured Description,Description,...,Media Link.8,dp_title,dp_description,dp_location_lat,dp_location_long,dp_nature_of_this_incident,dp_verificiation_status,dp_media_source,dp_translation_status,dp_description_translation
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31289.0,Unstructured,archived,2017-10-20 08:20:03,2017-10-26 17:30:37,2017-10-20 08:20:03,9906.0,purposepro1,"And @HCNicHailey, @BobGodec @MarietjeSchaake i...",NaN,...,NaN,NaN,"And @HCNicHailey, @BobGodec @MarietjeSchaake i...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31290.0,Media Reports,published,2017-10-20 08:25:03,2017-10-29 17:47:16,2017-10-20 08:25:03,8832.0,GRono,NaN,NaN,...,NaN,Varsities close amid electoral crisis (Muliro ...,"""Varsities close amid electoral crisis. https:...",0.288794,34.764456,NaN,"Verification,No need for verification","Media Stream Source,Other","Translation,No need for translation",NaN
3,31291.0,Unstructured,archived,2017-10-20 08:25:03,2017-10-26 18:33:01,2017-10-20 08:25:03,1265.0,dailynation,Varsities close amid electoral crisis. https:/...,NaN,...,NaN,NaN,Varsities close amid electoral crisis. https:/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31292.0,Unstructured,archived,2017-10-20 08:30:03,2017-10-26 11:39:42,2017-10-20 08:30:03,4585.0,AizakMohamed,After keeping every rigging mechanism in place...,NaN,...,NaN,NaN,After keeping every rigging mechanism in place...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
uchaguzi_data_df = uchaguzi_df[uchaguzi_df['Survey'].isin(labels)]

In [20]:
uchaguzi_rerun_df = uchaguzi_rerun_df[uchaguzi_rerun_df['Survey'].isin(labels)]

In [26]:
uchaguzi_rerun_df.shape

(7616, 213)

In [23]:
uchaguzi_rerun_df[uchaguzi_rerun_df['Survey'] != 'Unstructured'].shape

(781, 213)

In [24]:
uchaguzi_data_df[uchaguzi_data_df['Survey'] != 'Unstructured'].shape

(2413, 222)

In [ ]:
labelled_test = uchaguzi_data_df[]

In [30]:
uchaguzi_data_df['Media Stream Source'].unique()

array([nan, 'Media Stream Source,Twitter',
       'Media Stream Source,Mainstream Media,Twitter',
       'Media Stream Source,Facebook',
       'Media Stream Source,Facebook,Twitter',
       'Media Stream Source,Mainstream Media',
       'Media Stream Source,Twitter,Video',
       'Media Stream Source,Mainstream Media,Facebook,Twitter'],
      dtype=object)

In [25]:
uchaguzi_data_df.shape

(49106, 222)

In [ ]:
# 2017 label 
cols_2017 = ["Post ID", "Survey", "Post Status", "Created (UTC)", "Updated (UTC)", "Post Date (UTC)"] + deduplicated_cols

In [ ]:
cols_2017

In [ ]:
filtered_2017_df = uchaguzi_data_df[["Post ID","Survey","dp_description","Post Status", "Post Date (UTC)","dp_nature_of_this_incident","dp_location_lat","dp_location_long"]]

In [ ]:
filtered_rerun_df = uchaguzi_rerun_df[["Post ID","Survey","dp_description","Post Status", "Post Date (UTC)","dp_nature_of_this_incident","dp_location_lat","dp_location_long"]]
filtered_rerun_df.shape

In [ ]:
filtered_2017_df.columns

In [ ]:
filtered_2017_df.columns = ["id","annotation","text","status","date","categories","latitude","longitude"]
filtered_rerun_df.columns = ["id","annotation","text","status","date","categories","latitude","longitude"]

In [ ]:
filtered_2017_df = filtered_2017_df.dropna(subset='text')

In [ ]:
filtered_2017_df.shape

In [ ]:
filtered_2017_df.head()

In [ ]:
filtered_rerun_df = filtered_rerun_df.dropna(subset='text')

In [ ]:
filtered_df = filtered_df.dropna(subset=['dp_description'])

In [ ]:
filtered_df.columns = ["id","annotation","text","status","date","categories","latitude","longitude"]

In [ ]:
filtered_df.shape

In [ ]:
# Calculate number of duplicates
# Duplicate tweets that have different labels
duplicates = filtered_df.duplicated(["text"])
print(f"Duplicated election reports: {len(duplicates[duplicates == True])}")

duplicates = filtered_df.duplicated(["text","annotation"])
print(f"Duplicated with different election reports: {len(duplicates[duplicates == True])}")

# Count the number of tweets before dropping
print(f"The number of tweets before dropping duplicates is {len(filtered_df)}")

# # Drop the full duplicates
filtered_df = filtered_df.drop_duplicates(subset=["text"], keep='first')
print(f"The number of tweets after dropping duplicates is {len(filtered_df)}")

In [ ]:
duplicates = filtered_df.duplicated(["text"])
print(f"Duplicated election reports: {len(duplicates[duplicates == True])}")

In [ ]:
# Calculate number of duplicates
# Duplicate tweets that have different labels
duplicates = filtered_2017_df.duplicated(["text"])
print(f"Duplicated election reports: {len(duplicates[duplicates == True])}")

duplicates = filtered_2017_df.duplicated(["text","annotation"])
print(f"Duplicated with different election reports: {len(duplicates[duplicates == True])}")

# Count the number of tweets before dropping
print(f"The number of tweets before dropping duplicates is {len(filtered_2017_df)}")

# # Drop the full duplicates
filtered_2017_df = filtered_2017_df.drop_duplicates(subset=["text"], keep='first')
print(f"The number of tweets after dropping duplicates is {len(filtered_2017_df)}")

In [ ]:
# Calculate number of duplicates
# Duplicate tweets that have different labels
duplicates = filtered_rerun_df.duplicated(["text"])
print(f"Duplicated election reports: {len(duplicates[duplicates == True])}")

duplicates = filtered_rerun_df.duplicated(["text","annotation"])
print(f"Duplicated with different election reports: {len(duplicates[duplicates == True])}")

# Count the number of tweets before dropping
print(f"The number of tweets before dropping duplicates is {len(filtered_rerun_df)}")

# # Drop the full duplicates
filtered_rerun_df = filtered_rerun_df.drop_duplicates(subset=["text"], keep='first')
print(f"The number of tweets after dropping duplicates is {len(filtered_rerun_df)}")

In [ ]:
import json

In [ ]:
with open('2022_tweets.json', 'r') as file:
    data = file.readlines()

In [ ]:
json_data = [json.loads(line) for line in data]

In [ ]:
# Load all the tweets from the dataset
tweets_df = pd.DataFrame(json_data)

In [ ]:
# Get shape of tweet dataframe
tweets_df.shape

In [ ]:
filtered_tweets = tweets_df[["id","full_text"]]

In [ ]:
filtered_tweets

In [ ]:
def get_orig_tweet(row, tweets_df):
    source_id = int(row['Data Source ID'])
    tweet_text = pd.NA
    value = tweets_df.loc[tweets_df['id'] == source_id, 'full_text']
    if len(value) != 0:
        tweet_text = value.item()
    return tweet_text

In [ ]:
ushahidi_df.shape

In [ ]:
mask = (ushahidi_df['Survey'] == 'Unstructured') & (ushahidi_df['Source'] == 'twitter') 
unstructured_df = ushahidi_df[mask]

In [ ]:
unstructured_df.head()

In [ ]:
unstructured_df["dp_description"] = unstructured_df.apply(lambda row: get_orig_tweet(row, tweets_df), axis=1)

In [ ]:
unstructured_df["dp_description"].info()

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', None)

In [ ]:
unstructured_df[["dp_description"]].sample(20)

In [ ]:
unstructured_df.shape

In [ ]:
clean_unstructured = unstructured_df.dropna(subset=["dp_description"])

In [ ]:
clean_unstructured = clean_unstructured[["Post ID","Survey","dp_description","Post Status", "Post Date (UTC)","dp_nature_of_this_incident","dp_location_lat","dp_location_long"]]

In [ ]:
clean_unstructured.columns = ["id","annotation","text","status","date","categories","latitude","longitude"]

In [ ]:
# Calculate number of duplicates
# Duplicate tweets that have different labels
duplicates = clean_unstructured.duplicated(["text"])
print(f"Duplicated election reports: {len(duplicates[duplicates == True])}")

# Count the number of tweets before dropping
print(f"The number of tweets before dropping duplicates is {len(clean_unstructured)}")

# # Drop the full duplicates
clean_unstructured = clean_unstructured.drop_duplicates(subset=["text"], keep="first")
print(f"The number of tweets after dropping duplicates is {len(clean_unstructured)}")

In [ ]:
clean_unstructured.text.info()

In [ ]:
t = filtered_rerun_df[filtered_rerun_df.annotation != 'Unstructured']

In [ ]:
t

In [ ]:
filtered_df['Election Date'] = 'August 2022'
clean_unstructured['Election Date'] = 'August 2022'
filtered_2017_df['Election Date'] = 'August 2017'
filtered_rerun_df['Election Date'] = 'October 2017'

In [ ]:
combined_df = pd.concat([filtered_df, clean_unstructured, filtered_2017_df, filtered_rerun_df])

In [ ]:
combined_df = combined_df.rename(columns={"annotation":"label"})

In [ ]:
combined_df.info()

In [ ]:
combined_df.label.unique()

In [ ]:
combined_df.categories.unique()

In [ ]:
combined_df.shape

In [ ]:
combined_df[combined_df.status == 'archived']

In [ ]:
combined_df['text'].info()

In [ ]:
csv_df = combined_df.copy()

In [ ]:
csv_df = csv_df.reset_index(drop=True)

In [ ]:
import csv

In [ ]:
csv_df.to_csv("combined_election_reports.csv",  quoting=csv.QUOTE_NONNUMERIC, index=False)

In [ ]:
csv_df.shape

In [ ]:
reloaded_df = pd.read_csv('combined_election_reports.csv')

In [ ]:
reloaded_df.shape

In [ ]:
reloaded_df.head()

In [ ]:
combined_df

In [ ]:
combined_df = combined_df.rename(columns={'label': 'annotation'})

In [ ]:
dataset_rg = rg.read_pandas(reloaded_df, task="TextClassification")

In [ ]:
rg.log(dataset_rg, name='new_round')

In [ ]:
filtered_2017_df.shape

In [ ]:
check_labelled_2017 = filtered_2017_df[filtered_2017_df['label'] != 'unstructured_df']